Приложение 2 к отчету по теме "Анализ влияния параметров многофазного потока на механическую устойчивость трубопровода в условиях Новопортовского месторождения "

# Динамический анализ влияния многофазного потока на устойчивость трубопровода (анализ элемента трубопровода)

Файл с анализом результатов массовых расчетов режима работы трубопровода в OLGA

версия 0.5 (Шабонас Артур, Хабибуллин Ринат) от 09.10.2017 

Сравнение результатов расчета на динамическом многофазном симуляторе для двух вариантов элемента трубопровода длиной 1500 м. Рассматриваются модели трубопровода аналогичного трубопроводу Новопортовского месторождения по геометрическим параметров и параметрам флюидов.
Данный файл является расчетным файлом и содержит программный код выполняющий расчет и построение графиков а также результаты расчета.


In [ ]:
""" импортируем библиотеки необходимые для проведения расчетов"""
import tplread as tp
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

##  Блок загрузки результатов расчетов
На динамическом симуляторе проведены параметрические расчеты. Результаты расчетов записываются в файл. Один файл соответсвует одному расчету - набору дебитов жидкости, газа и давления в элементе трубопровода. Читаем данные из каталога с результатам расчетов. 

Загружается два вида моделей:
* первая модель описывает участок трубопровода с одним "горбом" и длинным горизонтальным участком после него
![схема модели](pic\model_simple.png "схема простой модели с одним горбом")

* вторая модель описывает участок трубопровода без "горба" - плоский
![схема модели](pic\model_plain.png "схема простой модели с одним горбом")

Обе модели содержат одинаковое количество контрольных точек в которых контролируются параметры потока. Обе модели запускались для различных расходов газа, жидкости и различных значений давления на конце трубопровода. 

Задачи моделей можно сформулировать следующим образом: 
* оценить условия образования больших пробок при различных условиях работы трубопровода
* оценить степень влияния пробок на механическую устойчивость трубопровода
* оценить влияния "горба" на образование пробок, сравнив режимы работы в одной точке при наличии и отсутствии "горба"

In [ ]:
# читаем сначала данные плоской модели
tsim = tp.TplParams('NPsimple/')
tsim.read_data()
tsim.calc_data()
tsim.name = 'модель плоская. '
# затем читаем данные модели с одним горбом модели
tpl = tp.TplParams('NPtest1/')
tpl.read_data()
tpl.calc_data()
tpl.name = 'модель с "горбом". '

#print(t.df_super)

## Служебный блок. Проверка считанных данных. 
Проверяются заголовки ключевых слов, диапазоны изменения параметров. 

In [ ]:
def check(t):
    # процедура вывод результатов проверки загрузки данных
    print(" файлов прочитано", len(t.files))
    print('ключевые слова для расчетов      ',t.key_list)
    print('полный список ключевых слов      ',t.key_list_all)
    print('список точек контроля параметров ',t.pipe_list)
    print('список значений давления в трубе ',t.p_end_list)
    print('список значений дебитов жидкости ',t.qliq_list)
    print('список значений дебитов газа     ',t.qgas_list)
    
check(tsim)     # проверим первый блок файлов и выведем результаты

In [ ]:
check(tpl) # проверим второй блок файлов и выведем результаты

## Соответствие приведенный названий точек контроля и их положения на трубе приведено на рисунке
![схема модели](pic/model_simple.svg "схема простой модели с одним горбом")

# Карты влияния потока на устойчивость трубопровода

Для всех расчитанных параметров можно построить карты влияния потока на устойчивость трубопровода. 
Карта показывает значения коэффциента устойчивости от расхода газа и жидкости, а также отношения коэффициента для "горбатой" трубы по отношению к трубе плоской

In [ ]:
"""выбираем значения параметров для анализа"""
def plot_trends_t(pp_num = 0, pend_num = 0, qg_num=1, ql_num=8  ):
#    qg=1           # номер значения дебита газа
#    ql=8           # номер значения дебита жидкости
#    pp=pp_num           # номер значения дебита участка трубы
#    p=1            # номер значения дебита давления
    k=['HOL','HOLEXP']   # номер значения ключевого слова
    print('===== истинное содержание жидкости в потоке ==========')
    tp.plot_trend_example(tsim,tpl,k,qg_num,ql_num,pp_num, pend_num)
    print('===== коэффициент механического влияния на компенсатор ==========')
    tp.plot_trend_super_example(tsim,tpl,qg_num,ql_num,pp_num,pend_num)
    k=['LSLEXP','LSBEXP']   # номер значения ключевого слова
    print('===== длина пробок и расстояние между пробками ==========')
    tp.plot_trend_example(tsim,tpl,k,qg_num,ql_num,pp_num,pend_num)
    k=['USG','USL']   # номер значения ключевого слова
    print('===== скорости движения жидкости и газа ==========')
    tp.plot_trend_example(tsim,tpl,k,qg_num,ql_num,pp_num,pend_num)
    k=['USFEXP','USTEXP']   # номер значения ключевого слова
    print('===== скорости фронта и хвоста пробки ==========')
    tp.plot_trend_example(tsim,tpl,k,qg_num,ql_num,pp_num,pend_num)



# Построение карт влияния потока на трубопровод

Ниже строятся карты для всех контрольных точек трубопровода

In [ ]:
i=0
for pp in tsim.pipe_list:           # строим для всех контрольных точек
    j=0
    for pend in tsim.p_end_list:
        print('\n\n ==========================================================================================================',
              '\n Результаты расчета параметров трубопровода: '
              '\n - контрольная точка - \"', pp,
              '\"\n - даление на конце участка - \"', pend, 
              '\n\n ==========================================================================================================')
        (amax,ql1,qg1,bmax, ql2,qg2) = tp.plot_map(tsim,tpl,i, pend=pend, val='mech',titl=' Мех.влияние', fmt = '.1f')
        tp.plot_map(tsim,tpl,i, pend=pend, val = 'slug_velocity', vm = 40, titl=' Скорости пробок', fmt = '.0f')
        tp.plot_map(tsim,tpl,i, pend=pend, val = 'slug_delta_holdup', vm = 0.3, titl=' Вес пробок', fmt = '.2f')
        tp.plot_map(tsim,tpl,i, pend=pend, val = 'slug_length', vm = 200 , titl=' Длина пробок', fmt = '.0f')
        tp.plot_map(tsim,tpl,i, pend=pend, val = 'bubble_length', vm = 1000, titl = ' Расстояния между пробками', fmt = '.0f')
        print("Максимальное значение на карте \"плоского трубопровода\" = ", bmax, ", при:"
                                                                                    "\n - скорости жидкости = ", ql2,
              "\n - скорости газа = ", qg2)
        print("\nМаксимальное значение на карте трубопровода с \"горбом\" = ", amax, ", при:"
                                                                                      "\n - скорости жидкости = ", ql1,
              "\n - скорости газа = ", qg1)
        plot_trends_t(pp_num= i,pend_num=j,qg_num=qg1,ql_num=ql1)
        if (qg1 != qg2) or (ql1 != ql2):
            plot_trends_t(pp_num= i,pend_num=j,qg_num=qg2,ql_num=ql2)
        j=j+1
    i=i+1

Анализ построенных карт показывает, что наибольшее влияния потока на трубу происходит на точках Pipe-7 - Pipe 13
![схема модели](pic/model_simple_1.svg "схема простой модели с одним горбом")
При этом 500 метром горизонтальной трубы практически выравнивают матрицы влияния

# Детальное рассмотрение отдельных вариантов

Рассмотрим детальные графики параметров для отдельных переменных, на основании которых проводился расчет карт влияния

## Контрольная точка Pipe-101
Рассмотрим карты влияния при давлении 10 атм

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, titl=' Мех.влияние')

Видно, что непосредственно перед горбом уже наблюдается повышенная механическая нагрузка на компенсатор. Усилия выросли примерно в 2 раза (синяя карта)

Рассмотрим карты влияния при давлении 5 атм

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, titl=' Мех.влияние')

Картина качественно не поменялась. 

Рассмотрим карты скоростей движения пробок

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, val = 'slug_velocity', vm = 40, titl=' Скорости пробок')

Видно, что для модели скорости выше на 20% - 40%. Чем больше скорость газа - тем больше скорости движения пробок жидкости. Перед разрушением (переходом на режим расслоенного потока) скорость достигают 30 м/сек.

Рассмотрим карты "веса пробок" - разности веса пробок и пузырей между пробками

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, val = 'slug_delta_holdup', vm = 0.3, titl=' Вес пробок')

Видно, что вес пробок снижается при росте скорости газа (чем больше газа тем меньше жидкости и в пробках и в пузырях между пробками)

Построим карты средних длин пробок

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, val = 'slug_length', vm = 200 , titl=' Длина пробок')

In [ ]:
tp.plot_map(tsim,tpl,1,pend=10, val = 'bubble_length', vm = 1000, titl = ' Расстояния между пробками')